###Atualização do pip

In [1]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


###Instalação do Apache Beam

In [2]:
pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.1/508.1 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.7/186.7 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424

###Importação das Bibliotecas

In [1]:
import apache_beam as beam

###Pipeline Derivados de Petróleo

In [3]:
# Tabela retornando o produto, estado e valor venda total

ufs = [
 'AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO'
 ]
 
produtos = [
          'Etanol Hidratado', 
          'GLP', 
          'Gasolina C', 
          'Gasolina de Aviação', 
          'Querosene Iluminante', 
          'Querosene de Aviação', 
          'Óleo Combustível', 
          'Óleo Diesel', 
]


for j in range(len(ufs)):
  for i in range(len(produtos)):
    pij = beam.Pipeline()
    pipe1 = (
            pij
            |f'Extrair do CSV{i}{j}'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
            |f'Separador do CSV{i}{j}'>> beam.Map(lambda record: record.split(','))
            |f'Filtrar derivado{i}{j}'>> beam.Filter(lambda record: (record[2] == produtos[i] and (record[1])==ufs[j]))
            |f'Agregar as Colunas{i}{j}'>> beam.Map(lambda record: (record[1],round(float(record[4]))))
            |f'Construir uma nova tabela{i}{j}'>> beam.CombinePerKey(sum)
            |f'1Construir uma nova tabela de derivados de petróleo{i}{j}'>> beam.Distinct()
            
        )

    pipe2 = (
            pij
            |f'{i}{j}Extrair do CSV'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
            |f'{i}{j}Separador do CSV'>> beam.Map(lambda record: record.split(','))
            |f'{i}{j}Filtrar derivados petrol'>> beam.Filter(lambda record: (record[2] == produtos[i] and (record[1])==ufs[j]))
            |f'{i}{j}Agregar as Colunas'>> beam.Map(lambda record: (record[1],record[2]))
            |f'{i}{j}Construir uma nova tabela de derivados petrol'>> beam.Distinct()
          
        )

    tabela = (
        {'Produto':pipe2, 'Total de venda em m3':pipe1 }
        |f'{i}{j}Agrupar as pernas' >> beam.CoGroupByKey()
        |f'{i}{j}Resultado'>> beam.Map(print)
      
    )


    pij.run()

('AC', {'Produto': ['Etanol Hidratado'], 'Total de venda em m3': [130249]})
('AC', {'Produto': ['GLP'], 'Total de venda em m3': [667]})
('AC', {'Produto': ['Gasolina C'], 'Total de venda em m3': [2144]})
('AC', {'Produto': ['Gasolina de Aviação'], 'Total de venda em m3': [3715]})
('AC', {'Produto': ['Querosene de Aviação'], 'Total de venda em m3': [100879]})
('AC', {'Produto': ['Óleo Combustível'], 'Total de venda em m3': [1470]})
('AC', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [3726]})
('AL', {'Produto': ['Etanol Hidratado'], 'Total de venda em m3': [1109]})
('AL', {'Produto': ['GLP'], 'Total de venda em m3': [3555]})
('AL', {'Produto': ['Gasolina C'], 'Total de venda em m3': [6692]})
('AL', {'Produto': ['Gasolina de Aviação'], 'Total de venda em m3': [1247]})
('AL', {'Produto': ['Querosene de Aviação'], 'Total de venda em m3': [5545]})
('AL', {'Produto': ['Óleo Combustível'], 'Total de venda em m3': [45231]})
('AL', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [78

###Pipeline Diesel

In [4]:
# Tabela retornando produto diesel, por estado e venda total

ufs = [
 'AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO'
 ]
 
for j in range(len(ufs)):
  
  pij = beam.Pipeline()
  pipe1 = (
          pij
          |f'Extrair do CSV{i}{j}'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
          |f'Separador do CSV{i}{j}'>> beam.Map(lambda record: record.split(','))
          |f'Filtrar diesel{i}{j}'>> beam.Filter(lambda record: (record[2] == 'Óleo Diesel' and (record[1])==ufs[j]))
          |f'Agregar as Colunas{i}{j}'>> beam.Map(lambda record: (record[1],round(float(record[4]))))
          |f'Construir uma nova tabela{i}{j}'>> beam.CombinePerKey(sum)
          |f'1Construir uma nova tabela diesel{i}{j}'>> beam.Distinct()
          
      )

  pipe2 = (
          pij
          |f'{i}{j}Extrair do CSV'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
          |f'{i}{j}Separador do CSV'>> beam.Map(lambda record: record.split(','))
          |f'{i}{j}Filtrar derivado diesel'>> beam.Filter(lambda record: (record[2] == 'Óleo Diesel' and (record[1])==ufs[j]))
          |f'{i}{j}Agregar as Colunas'>> beam.Map(lambda record: (record[1],record[2]))
          |f'{i}{j}Construir uma nova tabela de derivado diesel'>> beam.Distinct()
          
      )

  tabela = (
      {'Produto':pipe2, 'Total de venda em m3':pipe1 }
      |f'{i}{j}Agrupar as pernas' >> beam.CoGroupByKey()
      |f'{i}{j}Resultado'>> beam.Map(print)
      
  )


  pij.run()

('AC', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [3726]})
('AL', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [7858]})
('AM', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [20496]})
('AP', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [4992]})
('BA', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [60402]})
('CE', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [18889]})
('DF', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [8620]})
('ES', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [21832]})
('GO', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [50493]})
('MA', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [23316]})
('MG', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [138480]})
('MS', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [26768]})
('MT', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [51543]})
('PA', {'Produto': ['Óleo Diesel'], 'Total de venda em m3': [40984]})
('PB', {'Produto': ['Ól